In [1]:
!pip install ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.10.2-py3-none-any.whl size=29103 sha256=0fa1d57b401834b64fb048124a6a93177a76dfd9757be5c3aae7119afbfdac90
  Stored in directory: /root/.cache/pip/wheels/47/51/06/380dc516ea78621870b93ff65527c251afdfdc5fa9d7f4d248
Successfully built ta


In [2]:
!pip install --upgrade ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from ta.momentum import StochasticOscillator, RSIIndicator
from ta.trend import MACD, EMAIndicator
from ta.volatility import BollingerBands
from ta.volume import OnBalanceVolumeIndicator, ChaikinMoneyFlowIndicator, VolumeWeightedAveragePrice

In [4]:
pip install quandl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import quandl
import pandas as pd
import numpy as np

In [6]:
quandl.ApiConfig.api_key = "9b5_8yf5qojnDzTp74kG"

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# Prepare BSE Index 
bse_sensex = quandl.get("BSE/SENSEX", start_date='2004-05-01', end_date='2019-03-31')
bse_sensex['BSE_Close'] = bse_sensex['Close'].copy()
bse_sensex.drop(columns=['Open','High','Low','Close'], inplace = True)

# TODO: Load and prepare S&P here same as above

In [11]:
def addIndicatorsTargetAndNormalize(stock):
   fileName = stock + 'Stock.csv'
   df = pd.read_csv('/content/drive/MyDrive/StocksDatasets/' + fileName)
   df.set_index('Date', inplace=True)
   df.index = pd.to_datetime(df.index)
   
   # Merge BSE and stock datasets together
   merge = pd.merge(df,bse_sensex, how='inner', left_index=True, right_index=True)
   
   # TODO: Merge 'merge' DF and 'S&P' df here
   
   # Normalize the columns
   scaler = StandardScaler()
   merge[merge.columns] = scaler.fit_transform(merge)
   
   # Creating Target
   stock_data = merge.copy()
   stock_data.loc[:,'Close_shifted'] = stock_data['Close'].shift(-10)
   stock_data['Label'] = (stock_data['Close_shifted'] > stock_data['Close']).apply(lambda x: 1 if x else -1)
   merge['Label']= stock_data['Label']
   merge.drop(df.tail(10).index, inplace=True)

   # Remove rows with null values
   merge.dropna(inplace=True)

   merge.to_csv('/content/drive/MyDrive/StocksDatasets/WithIndicators/' + fileName)

In [13]:
def createIndicators(stock):
  print('Processing' + stock)
  fileName = stock + 'Stock.csv'
  df = pd.read_csv('/content/' + fileName)
  df.set_index('Date', inplace=True)
  df['Volume'] = df['No. of Shares'].copy()
  df.drop(columns=['No. of Shares'])

  # Calculate the Stochastic Oscillator
  so = StochasticOscillator(df['High'], df['Low'], df['Close'], window=14)
  so_k = so.stoch()
  so_d = so.stoch_signal()

  # Calculate the RSI
  rsi = RSIIndicator(df['Close'], window=14)

  # Calculate the Money Flow Index
  mfi = ChaikinMoneyFlowIndicator(df['High'], df['Low'], df['Close'], df['Volume'], window=14)

  # Calculate the MACD
  macd = MACD(df['Close'], window_slow=26, window_fast=12, window_sign=9)
  macd_signal = macd.macd_signal()
  macd_diff = macd.macd_diff()

  # Calculate the EMA
  ema = EMAIndicator(df['Close'], window=14)

  # Calculate the Triple EMA
  tema = EMAIndicator(EMAIndicator(EMAIndicator(df['Close'], window=14).ema_indicator(), window=14).ema_indicator(), window=14)

  # Calculate the Moving Average Convergence Divergence (MACD)
  macd = MACD(df['Close'], window_slow=26, window_fast=12, window_sign=9)
  macd_signal = macd.macd_signal()
  macd_diff = macd.macd_diff()

  # Calculate the Volume Weighted Average Price (VWAP)
  #vwap = df.eval('(Volume * (High + Low + Close) / 3).cumsum() / df.Volume.cumsum()')
  vwap = VolumeWeightedAveragePrice(df['High'], df['Low'], df['Close'], df['Volume'], window=14)
  # Calculate the On Balance Volume (OBV)
  obv = OnBalanceVolumeIndicator(df['Close'], df['Volume'])

  # Calculate the Bollinger Bands
  bbands = BollingerBands(df['Close'], window=14, window_dev=2)
  bbands_upper = bbands.bollinger_hband()
  bbands_lower = bbands.bollinger_lband()

  # Calculate the Price Rate of Change (ROC) indicator
  roc = df['Close'].pct_change(periods=14)

  # Calculate the Chaikin Oscillator
  adl = ((2 * df['Close'] - df['High'] - df['Low']) / (df['High'] - df['Low'])) * df['Volume']
  chaikin = EMAIndicator(adl, window=3).ema_indicator() - EMAIndicator(adl, window=10).ema_indicator()

  df['rsi'] = rsi.rsi()
  df['stocastic_oscillator'] = so.stoch()
  df['mfi'] = mfi.chaikin_money_flow()
  df['macd'] = macd.macd()
  df['ema'] = ema.ema_indicator() 
  df['tema'] = tema.ema_indicator()
  df['vwap'] = vwap.volume_weighted_average_price()
  df['obv'] = obv.on_balance_volume()
  df['bbands_upper'] = bbands.bollinger_hband()
  df['bbands_lower'] = bbands.bollinger_lband()
  df['chaikin'] = chaikin

  df.to_csv('/content/drive/MyDrive/StocksDatasets/' + fileName)
  

In [ ]:
STOCKS = ['BOM500875','BOM532939','BOM524715','BOM532215','BOM532648','BOM532500','BOM500470','BOM500570','BOM500112','BOM532540','BOM500209','BOM500295','BOM532174','BOM500180','BOM500696','BOM500510','BOM500247','BOM500520','BOM532921','BOM532977','BOM500182','BOM507685','BOM500010','BOM532454','BOM500820','BOM500312','BOM532898','BOM532187','BOM533278','BOM532555','BOM570001']

for stock in STOCKS:
  createIndicators(stock)

In [12]:
STOCKS = ['BOM500875','BOM532939','BOM524715','BOM532215','BOM532648','BOM532500','BOM500470','BOM500570','BOM500112','BOM532540','BOM500209','BOM500295','BOM532174','BOM500180','BOM500696','BOM500510','BOM500247','BOM500520','BOM532921','BOM532977','BOM500182','BOM507685','BOM500010','BOM532454','BOM500820','BOM500312','BOM532898','BOM532187','BOM533278','BOM532555','BOM570001']

for stock in STOCKS:
  addIndicatorsTargetAndNormalize(stock)

<ipython-input-11-c562c29c7f16>:5: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df.index = pd.to_datetime(df.index)


## Test Code

In [ ]:
# Load the historical price data
df = pd.read_csv('/content/drive/MyDrive/StocksDatasets/BOM500010Stock.csv')
df.set_index('Date', inplace=True)

In [ ]:
df.index = pd.to_datetime(df.index)

<ipython-input-28-4d24419b1a6b>:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df.index = pd.to_datetime(df.index)


In [ ]:
df.sort_index()

,Open,High,Low,Close,WAP,No. of Shares,No. of Trades,Total Turnover,Deliverable Quantity,% Deli. Qty to Traded Qty,...,stocastic_oscillator,mfi,macd,ema,tema,vwap,obv,bbands_upper,bbands_lower,chaikin
Date,,,,,,,,,,,,,,,,,,,,,
2004-01-01,648.5,650.00,640.10,646.25,647.152381,23566,602,15250793,12373.0,52.50,...,NaN,NaN,NaN,NaN,NaN,NaN,23566,NaN,NaN,NaN
2004-01-03,619.0,650.00,617.10,640.00,634.452774,116694,1795,74036832,71851.0,61.57,...,44.388079,0.024429,-6.106852,640.743227,651.661327,649.346577,-402729,696.752291,603.333423,9878.670474
2004-01-04,645.0,674.00,635.00,667.25,655.302360,61731,1775,40452470,35119.0,56.89,...,92.663043,0.276923,0.187416,625.044291,627.338683,609.559367,-109767,652.697614,577.623815,13572.009428
2004-01-06,589.0,589.00,548.00,559.10,565.574585,87813,2354,49664801,40255.0,45.84,...,56.644737,-0.114600,-13.806687,577.002303,605.071674,559.215734,10646077,613.225224,523.474776,-24738.472873
2004-01-07,523.0,532.50,517.05,529.05,524.842576,336982,4705,176862501,221011.0,65.59,...,16.270270,-0.110093,-15.987379,552.902123,581.936874,551.404968,11054407,600.071000,514.414715,47927.124044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-01,1978.0,2006.85,1973.95,1987.65,1987.630000,67729,2277,134620179,18603.0,27.47,...,76.877101,-0.367038,19.036719,1968.898488,1935.226876,1981.394541,-31419980,2019.480139,1911.791290,-248510.269500
2019-11-02,1944.0,1958.40,1936.75,1948.00,1947.430000,45450,1163,88510492,13297.0,29.26,...,65.560641,0.195135,-0.383369,1959.585376,1963.888644,1936.738192,-30738908,2007.944801,1898.898056,-10725.346358
2019-11-03,1889.7,1903.60,1885.60,1897.30,1894.610000,66049,2154,125137190,22661.0,34.31,...,92.321755,0.091969,-15.392584,1881.590911,1915.785196,1874.551035,-30707024,1908.773383,1833.676617,16204.180907


In [ ]:
mergedDataset = pd.merge(df,bse_sensex, how='inner', left_index=True, right_index=True)

In [ ]:
mergedDataset.sort_index()

,Open,High,Low,Close,WAP,No. of Shares,No. of Trades,Total Turnover,Deliverable Quantity,% Deli. Qty to Traded Qty,...,mfi,macd,ema,tema,vwap,obv,bbands_upper,bbands_lower,chaikin,BSE_Close
Date,,,,,,,,,,,,,,,,,,,,,
2004-05-03,633.00,643.00,633.00,639.75,639.135515,26617,645,17011870,16228.0,60.97,...,-0.118737,-5.087766,639.658925,650.230067,642.010198,-529270,680.884196,603.022947,4.531816e+03,5584.99
2004-05-04,675.00,691.00,660.05,673.45,678.303203,62229,1316,42210130,33587.0,53.97,...,0.324068,7.073688,637.479045,626.335405,616.117630,-104880,680.150899,568.306244,-3.659086e+03,5647.15
2004-05-05,625.00,650.00,623.05,645.15,642.003438,11714691,2127,7520871901,54569.0,0.47,...,0.563132,-5.499372,623.399615,635.264218,637.326380,11141389,647.773624,593.047805,2.402660e+06,5686.19
2004-05-07,530.50,538.00,526.15,531.85,531.111804,88834,2234,47180786,63484.0,71.46,...,-0.041807,-16.202838,547.731817,578.317707,543.932046,11084684,585.590549,512.530880,2.206027e+04,5669.58
2004-05-10,606.00,650.00,606.00,645.45,642.807417,157714,2533,101379729,105207.0,66.71,...,0.075736,14.829618,620.625998,591.540415,618.107461,15075578,647.601489,594.284225,5.675176e+04,5555.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-03-25,1978.80,1978.80,1937.80,1945.75,1984.560000,757306,3091,1502919472,698123.0,92.19,...,-0.101439,16.761206,1942.351594,1908.466995,1932.438392,-31179310,2018.117325,1849.525532,-1.557823e+05,37808.91
2019-03-26,1948.00,1958.60,1931.00,1945.65,1944.250000,85919,2469,167047723,30906.0,35.97,...,-0.116744,15.438805,1942.791381,1910.056368,1936.051128,-31265229,2013.819300,1865.980700,-5.071029e+04,38233.41
2019-03-27,1959.90,1959.90,1906.75,1920.45,1928.600000,103461,2898,199535360,38432.0,37.15,...,-0.173222,12.216538,1939.812531,1911.779162,1938.227094,-31368690,2010.915361,1873.920353,-2.074885e+04,38132.88


In [ ]:
df.head()
df.to_csv()

,Open,High,Low,Close,WAP,No. of Shares,No. of Trades,Total Turnover,Deliverable Quantity,% Deli. Qty to Traded Qty,Spread H-L,Spread C-O
Date,,,,,,,,,,,,
2004-01-01,648.5,650.00,640.10,646.25,647.152381,23566.0,602.0,15250793.0,12373.0,52.50,9.9,-2.25
2004-01-02,647.0,659.00,642.00,652.50,647.343497,41357.0,1298.0,26772185.0,25290.0,61.15,17.0,5.50
2004-01-05,650.0,664.00,641.00,657.35,651.945799,58228.0,1297.0,37961500.0,32734.0,56.22,23.0,7.35
2004-01-06,657.0,674.95,646.05,649.85,660.754778,64880.0,1405.0,42869770.0,36599.0,56.41,28.9,-7.15
2004-01-07,650.0,650.00,633.00,636.45,641.314528,665814.0,909.0,426996191.0,27562.0,4.14,17.0,-13.55


In [ ]:


# Load the historical price data
df = pd.read_csv('/content/BOM500010Stock.csv')
df.set_index('Date', inplace=True)

# Calculate the Stochastic Oscillator
so = StochasticOscillator(df['High'], df['Low'], df['Close'], window=14)
so_k = so.stoch()
so_d = so.stoch_signal()

# Calculate the RSI
rsi = RSIIndicator(df['Close'], window=14)

# Calculate the Money Flow Index
mfi = ChaikinMoneyFlowIndicator(df['High'], df['Low'], df['Close'], df['Volume'], window=14)

# Calculate the MACD
macd = MACD(df['Close'], window_slow=26, window_fast=12, window_sign=9)
macd_signal = macd.macd_signal()
macd_diff = macd.macd_diff()

# Calculate the EMA
ema = EMAIndicator(df['Close'], window=14)

# Calculate the Triple EMA
tema = EMAIndicator(EMAIndicator(EMAIndicator(df['Close'], window=14).ema_indicator(), window=14).ema_indicator(), window=14)

# Calculate the Moving Average Convergence Divergence (MACD)
macd = MACD(df['Close'], window_slow=26, window_fast=12, window_sign=9)
macd_signal = macd.macd_signal()
macd_diff = macd.macd_diff()

# Calculate the Volume Weighted Average Price (VWAP)
#vwap = df.eval('(Volume * (High + Low + Close) / 3).cumsum() / df.Volume.cumsum()')
vwap = VolumeWeightedAveragePrice(df['High'], df['Low'], df['Close'], df['Volume'], window=14)
# Calculate the On Balance Volume (OBV)
obv = OnBalanceVolumeIndicator(df['Close'], df['Volume'])

# Calculate the Bollinger Bands
bbands = BollingerBands(df['Close'], window=20, window_dev=2)
bbands_upper = bbands.bollinger_hband()
bbands_lower = bbands.bollinger_lband()

# Calculate the Price Rate of Change (ROC) indicator
roc = df['Close'].pct_change(periods=14)

# Calculate the Chaikin Oscillator
adl = ((2 * df['Close'] - df['High'] - df['Low']) / (df['High'] - df['Low'])) * df['Volume']
chaikin = EMAIndicator(adl, window=3).ema_indicator() - EMAIndicator(adl, window=10).ema_indicator()


In [ ]:
df.head()

,Open,High,Low,Close,WAP,Volume,No. of Trades,Total Turnover,Deliverable Quantity,% Deli. Qty to Traded Qty,Spread H-L,Spread C-O
Date,,,,,,,,,,,,
01-01-2004,648.5,650.00,640.10,646.25,647.152381,23566,602,15250793,12373.0,52.50,9.9,-2.25
02-01-2004,647.0,659.00,642.00,652.50,647.343497,41357,1298,26772185,25290.0,61.15,17.0,5.50
05-01-2004,650.0,664.00,641.00,657.35,651.945799,58228,1297,37961500,32734.0,56.22,23.0,7.35
06-01-2004,657.0,674.95,646.05,649.85,660.754778,64880,1405,42869770,36599.0,56.41,28.9,-7.15
07-01-2004,650.0,650.00,633.00,636.45,641.314528,665814,909,426996191,27562.0,4.14,17.0,-13.55


In [ ]:
df['rsi'] = rsi.rsi()
df['stocastic_oscillator'] = so.stoch()
df['mfi'] = mfi.chaikin_money_flow()
df['macd'] = macd.macd()
df['ema'] = ema.ema_indicator() 
df['tema'] = tema.ema_indicator()
df['vwap'] = vwap.volume_weighted_average_price()
df['obv'] = obv.on_balance_volume()
df['bbands_upper'] = bbands.bollinger_hband()
df['bbands_lower'] = bbands.bollinger_lband()
df['chaikin'] = chaikin

In [ ]:
df['bbands_lower'].head(50)

Date
01-01-2004           NaN
02-01-2004           NaN
05-01-2004           NaN
06-01-2004           NaN
07-01-2004           NaN
08-01-2004           NaN
09-01-2004           NaN
12-01-2004           NaN
13-01-2004           NaN
14-01-2004           NaN
15-01-2004           NaN
16-01-2004           NaN
19-01-2004           NaN
20-01-2004           NaN
21-01-2004           NaN
22-01-2004           NaN
23-01-2004           NaN
27-01-2004           NaN
28-01-2004           NaN
29-01-2004    619.298578
30-01-2004    619.354144
03-02-2004    619.378161
04-02-2004    619.623347
05-02-2004    619.623628
06-02-2004    620.261314
09-02-2004    620.035321
10-02-2004    619.067452
11-02-2004    618.790109
12-02-2004    617.906448
13-02-2004    617.737160
16-02-2004    618.357512
17-02-2004    619.143582
18-02-2004    619.203239
19-02-2004    618.809899
20-02-2004    622.410163
23-02-2004    626.208742
24-02-2004    623.327655
25-02-2004    622.852825
26-02-2004    618.684641
27-02-2004    611.81

In [ ]:
df.to_csv('testOutput.csv')